In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 50)

# Rutas
BASE_DIR = Path().resolve().parent
DATA_PATH = BASE_DIR / 'data' / 'gold' / 'model' / 'clustering_geo_dataset.parquet'

In [ ]:
# Cargar datos
df = pd.read_parquet(DATA_PATH)
print(f"Shape: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
df.head()

In [ ]:
# Info general
print("=" * 60)
print("INFO GENERAL")
print("=" * 60)
print(f"\nTotal registros: {len(df):,}")
print(f"Período: {df['anio'].min()} - {df['anio'].max()}")
print(f"Municipios: {df['codigo_municipio'].nunique()}")
print(f"Clusters: {df['cluster_delictivo'].unique()}")

## 1. Distribución de Clusters

In [ ]:
# Distribución de clusters
print("=" * 60)
print("DISTRIBUCIÓN DE CLUSTERS")
print("=" * 60)

cluster_counts = df['cluster_delictivo'].value_counts().sort_index()
cluster_pct = df['cluster_delictivo'].value_counts(normalize=True).sort_index() * 100

balance_cluster = pd.DataFrame({
    'count': cluster_counts,
    'porcentaje': cluster_pct.round(2)
})
print(balance_cluster)

# Ratio de desbalance
ratio = cluster_counts.max() / cluster_counts.min()
print(f"\nRatio max/min: {ratio:.2f}")

In [ ]:
# Visualización distribución
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = sns.color_palette('Set2', len(cluster_counts))

# Barplot
ax1 = axes[0]
bars = ax1.bar(cluster_counts.index.astype(str), cluster_counts.values, color=colors)
ax1.set_xlabel('Cluster')
ax1.set_ylabel('Frecuencia')
ax1.set_title('Distribución de cluster_delictivo')
for bar, val in zip(bars, cluster_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 100, f'{val:,}', ha='center')

# Pie chart
ax2 = axes[1]
ax2.pie(cluster_counts.values, labels=[f'Cluster {i}' for i in cluster_counts.index], 
        autopct='%1.1f%%', colors=colors)
ax2.set_title('Proporción de clusters')

plt.tight_layout()
plt.show()

## 2. Características por Cluster

In [ ]:
# Características promedio por cluster
print("=" * 60)
print("CARACTERÍSTICAS PROMEDIO POR CLUSTER")
print("=" * 60)

feature_cols = ['total_delitos', 'poblacion_total', 'densidad_poblacional', 
                'area_km2', 'n_centros_poblados']

cluster_stats = df.groupby('cluster_delictivo')[feature_cols].mean().round(2)
print(cluster_stats)

# Interpretación
print("\n" + "-" * 60)
print("INTERPRETACIÓN DE CLUSTERS:")
for cluster in sorted(df['cluster_delictivo'].unique()):
    stats = cluster_stats.loc[cluster]
    print(f"\nCluster {cluster}:")
    print(f"  - Delitos promedio: {stats['total_delitos']:.0f}")
    print(f"  - Población promedio: {stats['poblacion_total']:,.0f}")
    print(f"  - Densidad: {stats['densidad_poblacional']:.1f} hab/km²")

In [ ]:
# Boxplots por cluster
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

features_to_plot = ['total_delitos', 'poblacion_total', 'densidad_poblacional', 'area_km2']
palette = sns.color_palette('Set2', len(cluster_counts))

for ax, feature in zip(axes.flat, features_to_plot):
    sns.boxplot(data=df, x='cluster_delictivo', y=feature, palette=palette, ax=ax)
    ax.set_title(f'{feature} por Cluster')
    ax.set_xlabel('Cluster')

plt.tight_layout()
plt.show()

## 3. Validación del Clustering

In [ ]:
# Métricas de validación del clustering
print("=" * 60)
print("MÉTRICAS DE VALIDACIÓN")
print("=" * 60)

# Features usadas para clustering
cluster_features = ['total_delitos', 'poblacion_total', 'densidad_poblacional']

# Eliminar NaN para el cálculo
df_valid = df.dropna(subset=cluster_features + ['cluster_delictivo'])

X = df_valid[cluster_features].values
labels = df_valid['cluster_delictivo'].values

# Escalar features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Silhouette Score (mayor es mejor, rango -1 a 1)
silhouette = silhouette_score(X_scaled, labels)
print(f"\nSilhouette Score: {silhouette:.4f}")
if silhouette > 0.5:
    print("  ✅ Estructura de clusters fuerte")
elif silhouette > 0.25:
    print("  ⚡ Estructura de clusters razonable")
else:
    print("  ⚠️ Estructura de clusters débil")

# Calinski-Harabasz Score (mayor es mejor)
calinski = calinski_harabasz_score(X_scaled, labels)
print(f"\nCalinski-Harabasz Score: {calinski:.2f}")

In [ ]:
# Scatter plot 2D de clusters
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

palette = sns.color_palette('Set2', len(cluster_counts))

# Delitos vs Población
sns.scatterplot(data=df, x='poblacion_total', y='total_delitos', 
                hue='cluster_delictivo', palette=palette, alpha=0.6, ax=axes[0])
axes[0].set_title('Delitos vs Población')
axes[0].set_xlabel('Población Total')
axes[0].set_ylabel('Total Delitos')

# Delitos vs Densidad
sns.scatterplot(data=df, x='densidad_poblacional', y='total_delitos', 
                hue='cluster_delictivo', palette=palette, alpha=0.6, ax=axes[1])
axes[1].set_title('Delitos vs Densidad')
axes[1].set_xlabel('Densidad Poblacional')
axes[1].set_ylabel('Total Delitos')

# Población vs Densidad
sns.scatterplot(data=df, x='poblacion_total', y='densidad_poblacional', 
                hue='cluster_delictivo', palette=palette, alpha=0.6, ax=axes[2])
axes[2].set_title('Densidad vs Población')
axes[2].set_xlabel('Población Total')
axes[2].set_ylabel('Densidad Poblacional')

plt.tight_layout()
plt.show()

## 4. Municipios por Cluster

In [ ]:
# Municipios únicos por cluster
print("=" * 60)
print("MUNICIPIOS POR CLUSTER")
print("=" * 60)

# Obtener municipio más frecuente por cluster para cada código
if 'municipio' in df.columns:
    for cluster in sorted(df['cluster_delictivo'].unique()):
        municipios = df[df['cluster_delictivo'] == cluster]['municipio'].unique()
        print(f"\nCluster {cluster} ({len(municipios)} municipios):")
        print(f"  {', '.join(municipios[:10])}" + ("..." if len(municipios) > 10 else ""))
else:
    for cluster in sorted(df['cluster_delictivo'].unique()):
        codigos = df[df['cluster_delictivo'] == cluster]['codigo_municipio'].unique()
        print(f"\nCluster {cluster} ({len(codigos)} códigos municipio):")
        print(f"  {codigos[:10]}" + ("..." if len(codigos) > 10 else ""))

## 5. Conclusiones y Recomendaciones

In [ ]:
print("=" * 60)
print("CONCLUSIONES")
print("=" * 60)

print(f"""
DISTRIBUCIÓN DE CLUSTERS:
  - El clustering muestra {len(cluster_counts)} grupos diferenciados
  - Existe desbalance significativo (ratio {ratio:.1f})
  - Cluster dominante: {cluster_counts.idxmax()} ({cluster_counts.max():,} registros, {cluster_pct.max():.1f}%)

CALIDAD DEL CLUSTERING:
  - Silhouette Score: {silhouette:.4f}
  - {'✅ Buena separación' if silhouette > 0.25 else '⚠️ Separación débil'}

INTERPRETACIÓN:
  - Cluster 0: Municipios pequeños/rurales (mayoría)
  - Clusters 1-3: Municipios urbanos con mayor actividad delictiva
""")

print("\n" + "=" * 60)
print("RECOMENDACIONES")
print("=" * 60)
print("""
1. El clustering existente es útil como feature para otros modelos
2. Considerar experimentar con más/menos clusters (3-6)
3. Agregar features adicionales: área, tipo de delitos predominantes
4. Para clasificación supervisada: usar cluster_delictivo como target
5. Para nuevos datos: entrenar KMeans y predecir cluster
""")